In [ ]:
# Creating DataFrames

In [2]:
import json
import pandas as pd
from tqdm.notebook import tqdm


# Reading datasets

with open('../data/matches/matches_England.json') as f:
    matches_England = json.load(f)

with open('../data/players.json') as f:
    players = json.load(f)

with open('../data/teams.json') as f:
    teams = json.load(f)

with open('../data/events/events_England.json') as f:
    events_England = json.load(f)


## Matches DF

In [3]:
def find_side_and_score(match):
    # find the teams
    team0 = list(match["teamsData"].keys())[0]
    team1 = list(match["teamsData"].keys())[1]
    # check to see if team0 is home or away, add accordingly
    if match["teamsData"][team0]["side"] == "home":
        score0 = match["teamsData"][team0]["score"]
        score1 = match["teamsData"][team1]["score"]
        return (team0, team1, score0, score1)
    else:
        score0 = match["teamsData"][team0]["score"]
        score1 = match["teamsData"][team1]["score"]        
        return (team1, team0, score1, score0)


In [4]:
find_side_and_score(matches_England[0])

('1646', '1659', 1, 2)

In [9]:
matches = []
for match in matches_England:
    home_team, away_team, home_score, away_score = find_side_and_score(match)

    matches.append({
        'match_id': match['wyId'],
        'date': match['dateutc'],
        'home_team': int(home_team),
        'away_team': int(away_team),
        'home_score': int(home_score),
        'away_score': int(away_score),
        'label': match['label'],
        'venue': match['venue'],
        'gameweek': match['gameweek'],
        'winner_team_id': int(match['winner'])
    })
matches_df = pd.DataFrame(matches)

matches_df.to_csv('dataframes/matches.csv', index=False)


## Players DF

In [10]:
players_df = pd.DataFrame(players)
players_df = players_df.drop(["passportArea", "role", "birthArea"], axis=1)
players_df.head()

players_df.to_csv('dataframes/players.csv', index=False)

In [11]:
players_df.head()

,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,wyId,foot,shortName,currentNationalTeamId
0,78,Harun,,Tekin,4502,1989-06-17,187,32777,right,H. Tekin,4687
1,73,Malang,,Sarr,3775,1999-01-23,182,393228,left,M. Sarr,4423
2,72,Over,,Mandanda,3772,1998-10-26,176,393230,,O. Mandanda,null
3,82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,32793,right,A. N'Diaye,19314
4,84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,393247,right,I. Konat\u00e9,null


## Teams DF

In [12]:
teams_df = pd.DataFrame(teams)
teams_df = teams_df.drop('area', axis=1)
teams_df.head()
teams_df.to_csv('dataframes/teams.csv', index=False)

## Events DF

In [13]:
events_df = pd.DataFrame(events_England)
events_df = events_df.drop(['positions', 'tags'], axis=1)
events_df.head()
events_df.to_csv('dataframes/events.csv', index=False)

## Positions DF

In [14]:
positions_data = []

for event in events_England:
    ID = event["id"]
    initialX = event["positions"][0]["x"]
    initialY = event["positions"][0]["y"]
    
    try:
        finalX = event["positions"][1]["x"]
    except IndexError:
        finalX = None
        
    try:
        finalY = event["positions"][1]["y"]
    except IndexError:
        finalY = None
    positions_data.append({"id": ID, "initialX": initialX, "initialY": initialY, "finalX": finalX, "finalY": finalY})

positions_df = pd.DataFrame(positions_data)
positions_df.head()
positions_df.to_csv("dataframes/positions.csv", index=False)

## EventTags DF

In [15]:
unique_tags = []
events_df = pd.DataFrame(events_England)

event_tags = []

for index, event in events_df.iterrows():
    for tag in event["tags"]:
        event_tags.append({
            'event_id': event['id'],
            'tag': tag['id']
        })
event_tags_df = pd.DataFrame(event_tags)


In [16]:
event_tags_df.to_csv("dataframes/event_tags.csv", index=False)

## Passes Received

In [17]:
events_df.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175


In [25]:
# get indices of passes
pass_indices = events_df.index[events_df['eventId'] == 8]

pass_events = events_df[events_df['eventId'] == 8]

# get the rows right after each pass
events_after_pass = events_df.loc[pass_indices + 1]

data = {
    'pass_id': pass_events['id'],
    'pass_made_by': pass_events['playerId'],
    'pass_received_by': events_after_pass['playerId']
}

pass_df = pd.DataFrame(data)

In [26]:
pass_df

,pass_id,pass_made_by,pass_received_by
0,177959171.0,25413.0,NaN
1,177959172.0,370224.0,370224.0
2,177959173.0,3319.0,3319.0
3,177959174.0,120339.0,120339.0
4,177959175.0,167145.0,167145.0
...,...,...,...
643136,NaN,NaN,8561.0
643141,251596225.0,8561.0,NaN
643142,251596226.0,20620.0,20620.0
643143,251596229.0,14703.0,14703.0


In [23]:
events_after_pass

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175
5,8,Simple pass,[{'id': 1802}],3319,"[{'y': 88, 'x': 72}, {'y': 75, 'x': 77}]",2499719,Pass,1609,1H,12.548934,85,177959177
...,...,...,...,...,...,...,...,...,...,...,...,...
643135,8,Simple pass,[{'id': 1801}],379209,"[{'y': 96, 'x': 61}, {'y': 85, 'x': 40}]",2500098,Pass,1633,2H,2763.698599,85,251596221
643136,7,Touch,[],8561,"[{'y': 85, 'x': 40}, {'y': 84, 'x': 62}]",2500098,Others on the ball,1633,2H,2766.359485,72,251596222
643142,8,High pass,[{'id': 1801}],20620,"[{'y': 89, 'x': 53}, {'y': 72, 'x': 99}]",2500098,Pass,1633,2H,2785.702032,83,251596226
643143,8,Cross,"[{'id': 402}, {'id': 2101}, {'id': 1802}]",14703,"[{'y': 72, 'x': 99}, {'y': 0, 'x': 0}]",2500098,Pass,1633,2H,2789.232666,80,251596229
